In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.stats.mstats import gmean 
import pandas_market_calendars as mcal
import QuantLib as ql

# Control Variate Monte Carlo on energy futures, taking care of rollover

## Inputs:
Initial values of both futures: F10,F20 <br>
Volatility of both futures: sigma_1,sigma_2 <br>
Correlation between two futures: rho<br>
Risk free rate: r<br>
Rollover day in market days(in years): T1 e.g. one week would be 5/258 <br>
Maturity day: T2 <br>
First day of averaging: first_date <br>
Step size of the averaging: step = 1/258 by default <br>
Strike: K <br>
Type of option: Type 

### Closed-form Geometric Asian <br>
g = $log(G)$ ~ $N(\mu,\sigma^2)$

### BS Wrapper <br>
$S_0$ = $E(G_T)$ = $e^{\mu+\sigma^2/2}$  <br>
$K$ = $K$ <br>
$T$ = $T_2$ <br>
$r$ = $r$ <br>
$y$ = $r$ <br>
$\sigma$ = $\sqrt{\sigma^2/T}$ 

### Control Variate Monte Carlo

$A^* = E(A) + \beta (E(G) - G)$ <br>
$\beta = \frac{cov(A,G)}{var(G)}$ <br>
$SE_A* = \sqrt{\frac{var(A) - 2\beta cov(A,G) + \beta^2 var(G)}{n}}$ <br>

In [15]:
class Asian(object):
    step = 1/258
    def __init__(self,F10,F20,K,sigma_1,sigma_2,rho,r,today,first,rollover,mature,Type):
        self.F10 = F10
        self.F20 = F20
        self.K = K
        self.sigma_1 = sigma_1
        self.sigma_2= sigma_2
        self.rho = rho
        self.r = r
        self.first_date,self.T1,self.T2 = self.f_T(today,first,rollover,mature)        
        self.Type = Type

    def f_T(self,today,first,rollover,mature):
        '''
        Deal with market dates
        
        parameters:
        first_date: first date when pricing period starts
        T1: rollover date when the closing price of the next future is used instead of the first due to liquidity  
        T2: last day of pricing period +1, to make codes easier to read

        return:
        first_date,T1,T2 from today as fractions of a year
        '''

        ice = mcal.get_calendar('ICE')
        schedule_first = ice.schedule(today,first)
        l_first = len(schedule_first)
        schedule_rollover = ice.schedule(today,rollover)
        l_rollover = len(schedule_rollover)
        schedule_mature = ice.schedule(today,mature)
        l_mature = len(schedule_mature)

        if date.strftime(schedule_rollover.index[0],'%Y-%m-%d') == today:
            return (l_first-1)/258,(l_rollover-1)/258, (l_mature)/258 
                    # (l_mature)/258 is actually [the relative date of maturity +1] to accomodate to the pricing code
        else:
            return (l_first)/258,(l_rollover)/258, (l_mature+1)/258
                    # (l_mature+1)/258 is actually [the relative date of maturity +1] to accomodate to the pricing code


    @property
    def n(self):
        '''
        number of prices to take average on
        '''
        return int((self.T2 - self.first_date)/self.step)
    
    @property
    def mu(self):
        '''
        effective Mu in Geometric pricing
        '''
        return 1/self.n * ( ((self.T1 - self.first_date)*258)*np.log(self.F10)
                      -0.5*self.sigma_1**2 *np.sum(np.arange(int(self.first_date*258),int(self.T1*258)))/258
                      +(self.T2 - self.T1)*258*np.log(self.F20)
                      -0.5*self.sigma_2**2 *np.sum(np.arange(int(self.T1*258),int(self.T2*258)))/258 )

    @property
    def sigma(self):
        '''
        effective Sigma in Geometric pricing
        '''
        vol={0:sigma_1,1:sigma_2}
        return 1/self.n *( sum(
                      vol[i>(self.T1 - 1/258)]*vol[j>(self.T1 - 1/258)]*min(i,j)*self.rho**(int(min(i,j)<self.T1 and max(i,j) > (self.T1- 1/258)))
                      for i in np.arange(int(self.first_date*258),int(self.T2*258))/258 
                      for j in np.arange(int(self.first_date*258),int(self.T2*258))/258) )**0.5

    
    def log(func):
        '''
        wrapper functions to put effective parameters into BS model
        '''
        def wrapper(self):
            S0 =  np.exp(self.mu+self.sigma**2/2)
            y = self.r 
            sigma = (self.sigma**2/self.T2)**0.5
            return func(S0,self.K,self.T2-1/258,self.r,y,sigma,self.Type)
        return wrapper

    
    @log
    def BS(S0,K,T,r,y,sigma,Type):
        '''
        vanilla BS model
        '''

        d1 = (np.log(S0/K)+(r-y+0.5*sigma**2)*T)/(sigma*T**0.5)
        d2 = d1 - sigma*T**0.5
        if Type.upper() == 'CALL':
            
            return np.exp(-y*T)*S0*norm.cdf(d1) - np.exp(-r*T)*K*norm.cdf(d2)
        else:
            return -np.exp(-y*T)*S0*norm.cdf(-d1) + np.exp(-r*T)*K*norm.cdf(-d2)
    
    @property  
    def GeoMeanClosedForm(self):
        '''
        closed formed solution of geometric asian
        '''
        return self.BS()
    
    def f_payoff(self,ST):
        '''
        payoff function of call/put
        '''
        if self.Type.upper() == 'CALL':
            return max(ST-self.K,0)
        else:
            return max(self.K-ST,0)
    
    
    def ArithMeanCVMM(self,n_path):
        '''
        CVMM of arithmetic asian using geometric asian as control variate
        '''
        n_dots = int(self.T2 * 258.0) - 1
        if rho == 1:
            L = np.array([[1.00, 0.0],
               [1.00, 0.0]])
        elif rho == -1:
            L = np.array([[1.00, 0.0],
               [-1.00, 0.0]])
        elif rho == 0:
            L = np.array([[1.00, 0.0],
               [0.00, 1.00]])
        else:
            cor_matrix = np.array([[1.0,self.rho],
                                   [self.rho,1.0]])
            L = np.linalg.cholesky(cor_matrix)

        #generate correlated paths
        paths = L.dot(np.random.normal(0,1,(2,int(n_path*n_dots))))

        # scale by std
        paths = np.array([[self.step**0.5 * self.sigma_1,0],[0,self.step**0.5 * self.sigma_2]]) @ paths

        #add drift
        paths = (paths.T+[-0.5* self.sigma_1**2 * self.step, -0.5* self.sigma_2**2 * self.step]).T

        #cumsum each d(logF)
        for i in np.arange(0,int(n_path*n_dots), n_dots):
            paths[:,i:(i+n_dots)] = np.cumsum(paths[:,i:(i+n_dots)], axis = 1)

        #add initial price
        paths = (paths.T+[np.log(self.F10),np.log(self.F20)]).T

        #take to exponential
        paths = np.exp(paths)

        #calculate realized a-mean and g-mean for each path
        averaging_pos = list(zip(
                                 np.append(np.array([0]*int((self.T1-self.first_date)/self.step)),
                                           np.array([1]*int((self.T2-self.T1)/self.step))),
                                 np.append(np.arange(int(self.first_date/self.step)-1,int(self.T1/self.step)-1),
                                           np.arange(int(self.T1/self.step)-1,int(self.T2/self.step)-1))))


        a_sum = 0.0
        g_sum = 0.0
        a_2_sum = 0.0
        g_2_sum = 0.0
        ag_sum = 0
        
        diff = []

        for i in np.arange(0,int(n_path*n_dots), n_dots):

            temp = paths[:,i:(i+n_dots)]
            prices = [temp[int(pos[0]),int(pos[1])] for pos in averaging_pos]

            #a, a2
            a_mean = np.mean(prices)
            a_payoff = self.f_payoff(a_mean)
            a_value = np.exp(-self.r*self.T2) * a_payoff
            a_sum += a_value
            a_2_sum += a_value**2

            #g, g2
            g_mean = gmean(prices)
            g_payoff = self.f_payoff(g_mean)
            g_value = np.exp(-self.r*self.T2) * g_payoff
            g_sum += g_value
            g_2_sum += g_value**2

            #a*g
            ag_sum += a_value * g_value
            


        E_a = a_sum/n_path
        E_a_2 = a_2_sum/n_path
        E_g = g_sum/n_path
        E_g_2 = g_2_sum/n_path
        E_ag = ag_sum/n_path

        var_a = E_a_2 - E_a**2
        var_g = E_g_2 - E_g**2
        cov_ag = E_ag - E_a * E_g

        Beta = cov_ag / var_g
        E_a = E_a + Beta * (self.GeoMeanClosedForm - E_g)
        se_a = ((1/n_path) * (var_a - 2* Beta * cov_ag + Beta**2 * var_g))**0.5
        se_a_Beta1 = ((1/n_path) * (var_a - 2 * cov_ag +  var_g))**0.5
        se_a_BB = (var_a/n_path) ** 0.5
        return {'E_a':E_a,'Beta':Beta,'se_a':se_a,'se_a_Beta1':se_a_Beta1,'se_a_BB':se_a_BB}
        

In [16]:
F10 = 493.0
F20 = 492.750
sigma_1 = 0.015813514*258**0.5
sigma_2 = 0.015402859*258**0.5
rho = 0.99
r = -0.00628919
today = '2020-02-04'
first = '2020-04-01'
rollover = '2020-04-08'
mature = '2020-04-30'
K = 500
Type = 'call'

In [17]:
a=Asian(F10,F20,K,sigma_1,sigma_2,rho,r,today,first,rollover,mature,Type)

In [18]:
a.ArithMeanCVMM(1000)

{'E_a': 17.573129789257468,
 'Beta': 1.0041384234603037,
 'se_a': 0.005729019969855859,
 'se_a_Beta1': 0.0069060797979048775,
 'se_a_BB': 0.9357404528614592}